In [ ]:
# 導入所需函式庫
import json
import pandas as pd
from datetime import datetime
import os

print("函式庫導入完成")


In [ ]:
# 讀取JSON檔案
json_file = "codis_C0F9Y0_20160429_20250716.json"

print(f"正在讀取檔案: {json_file}")
print(f"檔案大小: {os.path.getsize(json_file) / 1024 / 1024:.2f} MB")

with open(json_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

print("JSON檔案讀取完成")
print(f"資料結構: {type(data)}")

# 檢查最外層結構
if isinstance(data, list) and len(data) > 0:
    first_item = data[0]
    print(f"第一筆資料的keys: {first_item.keys()}")
    print(f"code: {first_item.get('code')}")
    print(f"message: {first_item.get('message')}")
    print(f"data陣列長度: {len(first_item.get('data', []))}")
    
    # 檢查第一個測站的資料
    if first_item.get('data') and len(first_item['data']) > 0:
        station_data = first_item['data'][0]
        print(f"測站ID: {station_data.get('StationID')}")
        print(f"時間序列資料筆數: {len(station_data.get('dts', []))}")


In [ ]:
# 資料轉換函數
def flatten_weather_data(weather_record, station_id):
    """
    展平單筆氣象資料記錄
    """
    # 分離日期和時間
    datetime_str = weather_record.get('DataTime', '')
    if 'T' in datetime_str:
        date_part, time_part = datetime_str.split('T')
    else:
        date_part, time_part = '', ''
    
    # 基本資訊
    flattened = {
        'StationID': station_id,
        'date': date_part,
        'hour': time_part,
    }
    
    # 展平所有巢狀欄位
    fields_mapping = {
        'StationPressure': 'StationPressure_Instantaneous',
        'AirTemperature': 'AirTemperature_Instantaneous', 
        'RelativeHumidity': 'RelativeHumidity_Instantaneous',
        'WindSpeed': 'WindSpeed_Mean',
        'WindDirection': 'WindDirection_Mean',
        'PeakGust': ['PeakGust_Maximum', 'PeakGust_Direction', 'PeakGust_MaximumTime'],
        'Precipitation': ['Precipitation_Accumulation', 'Precipitation_MeltFlag'],
        'SunshineDuration': 'SunshineDuration_Total',
        'GlobalSolarRadiation': 'GlobalSolarRadiation_Accumulation',
        'SoilTemperatureAt0cm': 'SoilTemperatureAt0cm_Instantaneous',
        'SoilTemperatureAt5cm': 'SoilTemperatureAt5cm_Instantaneous',
        'SoilTemperatureAt10cm': 'SoilTemperatureAt10cm_Instantaneous',
        'SoilTemperatureAt20cm': 'SoilTemperatureAt20cm_Instantaneous',
        'SoilTemperatureAt50cm': 'SoilTemperatureAt50cm_Instantaneous',
        'SoilTemperatureAt100cm': 'SoilTemperatureAt100cm_Instantaneous'
    }
    
    # 處理每個欄位
    for source_field, target_fields in fields_mapping.items():
        source_data = weather_record.get(source_field, {})
        
        if isinstance(target_fields, list):
            # 處理多個子欄位的情況 (如 PeakGust, Precipitation)
            if source_field == 'PeakGust':
                flattened['PeakGust_Maximum'] = source_data.get('Maximum')
                flattened['PeakGust_Direction'] = source_data.get('Direction') 
                flattened['PeakGust_MaximumTime'] = source_data.get('MaximumTime')
            elif source_field == 'Precipitation':
                flattened['Precipitation_Accumulation'] = source_data.get('Accumulation')
                flattened['Precipitation_MeltFlag'] = source_data.get('MeltFlag')
        else:
            # 處理單一子欄位的情況
            if source_field in ['StationPressure', 'AirTemperature', 'RelativeHumidity']:
                flattened[target_fields] = source_data.get('Instantaneous')
            elif source_field in ['WindSpeed', 'WindDirection']:
                flattened[target_fields] = source_data.get('Mean')
            elif source_field == 'SunshineDuration':
                flattened[target_fields] = source_data.get('Total')
            elif source_field == 'GlobalSolarRadiation':
                flattened[target_fields] = source_data.get('Accumulation')
            elif 'SoilTemperature' in source_field:
                flattened[target_fields] = source_data.get('Instantaneous')
    
    return flattened

print("資料轉換函數定義完成")


In [ ]:
# 執行資料轉換
print("開始處理氣象資料...")

all_records = []
total_processed = 0

# 遍歷所有回應資料
for response_item in data:
    if response_item.get('code') == 200 and 'data' in response_item:
        # 遍歷每個測站
        for station_data in response_item['data']:
            station_id = station_data.get('StationID', '')
            
            # 遍歷該測站的所有時間序列資料
            for weather_record in station_data.get('dts', []):
                flattened_record = flatten_weather_data(weather_record, station_id)
                all_records.append(flattened_record)
                total_processed += 1
                
                # 每處理1000筆資料顯示進度
                if total_processed % 1000 == 0:
                    print(f"已處理 {total_processed} 筆資料...")

print(f"資料處理完成！總共處理了 {total_processed} 筆資料")

# 轉換為DataFrame
df = pd.DataFrame(all_records)
print(f"DataFrame 建立完成，形狀: {df.shape}")
print(f"欄位: {list(df.columns)}")


In [ ]:
# 資料預覽和基本資訊
print("=== 資料預覽 ===")
print("前5筆資料:")
print(df.head())

print("\n=== 資料基本資訊 ===")
print(f"總筆數: {len(df)}")
print(f"總欄位數: {len(df.columns)}")

print("\n=== 時間範圍 ===")
print(f"最早日期: {df['date'].min()}")
print(f"最晚日期: {df['date'].max()}")

print("\n=== 測站資訊 ===") 
unique_stations = df['StationID'].unique()
print(f"測站數量: {len(unique_stations)}")
print(f"測站代號: {unique_stations}")

print("\n=== 資料型態 ===")
print(df.dtypes)

print("\n=== 缺失值統計 ===")
missing_summary = df.isnull().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)
if len(missing_summary) > 0:
    print("有缺失值的欄位:")
    print(missing_summary)
else:
    print("沒有缺失值")


In [ ]:
# 輸出為CSV檔案
output_file = "codis_C0F9Y0_weather_data.csv"

print(f"正在輸出CSV檔案: {output_file}")

# 確保欄位順序
column_order = [
    'StationID', 'date', 'hour',
    'StationPressure_Instantaneous', 'AirTemperature_Instantaneous', 'RelativeHumidity_Instantaneous',
    'WindSpeed_Mean', 'WindDirection_Mean', 
    'PeakGust_Maximum', 'PeakGust_Direction', 'PeakGust_MaximumTime',
    'Precipitation_Accumulation', 'Precipitation_MeltFlag',
    'SunshineDuration_Total', 'GlobalSolarRadiation_Accumulation',
    'SoilTemperatureAt0cm_Instantaneous', 'SoilTemperatureAt5cm_Instantaneous', 
    'SoilTemperatureAt10cm_Instantaneous', 'SoilTemperatureAt20cm_Instantaneous',
    'SoilTemperatureAt50cm_Instantaneous', 'SoilTemperatureAt100cm_Instantaneous'
]

# 重新排列欄位順序
df_ordered = df[column_order]

# 輸出CSV
df_ordered.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"CSV檔案輸出完成！")
print(f"檔案名稱: {output_file}")
print(f"檔案大小: {os.path.getsize(output_file) / 1024:.2f} KB")

# 驗證輸出
print("\n=== 輸出驗證 ===")
test_df = pd.read_csv(output_file)
print(f"讀取驗證: {len(test_df)} 筆資料, {len(test_df.columns)} 個欄位")
print("CSV檔案前3筆資料:")
print(test_df.head(3))
